# A Quick Introduction to Modeling Optimization Problems with OptiGraphs.jl

## Load Packages

In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__,".."))
push!(LOAD_PATH,joinpath(@__DIR__,"../.."))
using Plasmo
using Ipopt

  Activating project at `~/git/Plasmo.jl/examples`
[ Info: Precompiling Plasmo [d3f7391f-f14a-50cc-bbe4-76a32d1bad3c]


## Create a new OptiGraph
Here we create a new optigraph and add one variable to it.

In [25]:
graph = OptiGraph(;name=:my_graph)
graph

An OptiGraph:  my_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         0                0
          Edges:         0                0
      Subgraphs:         0                0
      Variables:         0                0
    Constraints:         0                0


## Add OptiNodes
Optinodes extend the JuMP.AbstractModel and offer the same syntax used to construct a `JuMP.Model`. Here we add four nodes and create variables and constraints on each of them.

In [26]:
n1 = add_node(graph)
@variable(n1, 0 <= x <= 2)
@variable(n1, 0 <= y <= 3)
@variable(n1, 0 <= z <= 2)
@constraint(n1, x + y + z >= 4)

n2 = add_node(graph)
@variable(n2, x >= 0)
@constraint(n2, ref, exp(x) >= 2)
@variable(n2, 0 <= z <= 2)
@constraint(n2, z + x >= 4)

n3 = add_node(graph)
@variable(n3, x[1:3] >= 0)
@constraint(n3, nlcon, exp(x[3]) >= 5)
@constraint(n3, conref, sum(x[i] for i in 1:3) == 10)

n4 = add_node(graph)
@variable(n4, x[1:2] >= 0)
@constraint(n4, sum(x[i] for i in 1:2) >= 10)
@constraint(n4, ref, exp(x[2]) >= 4)

graph

An OptiGraph:  my_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         4                4
          Edges:         0                0
      Subgraphs:         0                0
      Variables:        10               10
    Constraints:        21               21


## Add OptiEdges (Linking Constraints)
Edges can be added to create coupling constraints between optinodes. Here we create a constraint that couples nodes `n1` and `n2`.

In [27]:
edge1 = add_edge(graph, n1, n2)
@constraint(graph, n1[:x] == n2[:x])

my_graph.n1.x - my_graph.n2.x = 0

We can also create linking constraints directly. This implicitly creates edges between nodes `n2` and `n3`, and `n3` and `n4`.

In [28]:
@linkconstraint(graph, link2, n2[:x] == n3[:x][3])
@linkconstraint(graph, link3, n3[:x][1] == n4[:x][1])
graph

An OptiGraph:  my_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         4                4
          Edges:         3                3
      Subgraphs:         0                0
      Variables:        10               10
    Constraints:        24               24


## Create an OptiGraph Objective
The graph objective can be defined over variables contained within its nodes. Here we pose a nonlinear objective function we wish to minimize.

In [29]:
# set an objective for the graph
@objective(graph, Min, n1[:y] + n2[:x] - (n3[:x][1] + n3[:x][2] + n3[:x][3]) + n4[:x][2]^3)

(my_graph.n1.y + my_graph.n2.x - my_graph.n3.x[1] - my_graph.n3.x[2] - my_graph.n3.x[3]) + (my_graph.n4.x[2] ^ 3)

## Solve with an Optimizer
Plasmo.jl optigraphs are compatible with all optimizers available through MathOptInterface.jl. Here we use Ipopt to solve our optigraph problem.

In [30]:
optimizer = Ipopt.Optimizer
set_optimizer(graph, optimizer)
optimize!(graph)

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        9
Number of nonzeros in inequality constraint Jacobian.:       10
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:       10
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -9.9989900e-03 9.98e+00 1.10e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## Query Solution
We lastly show how to query the values of the optimization solution. Plasmo.jl supports most (all) methods available in JuMP.jl. Here we query the objective value, the primal variable values, and all of the constraint dual values.

In [31]:
println("objective value = ", objective_value(graph))

println()
println("variable values:")
for var in all_variables(graph)
    println(var, " = ", value(var))
end

println()
println("constraint dual values:")
for constraint in all_constraints(graph)
    println("($constraint) = $(dual(constraint))")
end

objective value = -2.083829063082021e-6

variable values:
my_graph.n1.x = 1.9999999404261217
my_graph.n1.y = 5.6193499597211924e-9
my_graph.n1.z = 2.0000000169789374
my_graph.n2.x = 1.9999999404261217
my_graph.n2.z = 2.0000000197868992
my_graph.n3.x[1] = 8.000000069365052
my_graph.n3.x[2] = -9.791174667922972e-9
my_graph.n3.x[3] = 1.9999999404261217
my_graph.n4.x[1] = 8.000000069365052
my_graph.n4.x[2] = 1.9999998308437743

constraint dual values:
(my_graph.n1.x ≥ 0) = 0.0
(my_graph.n1.y ≥ 0) = 0.14723074499479147
(my_graph.n1.z ≥ 0) = 0.0
(my_graph.n2.x ≥ 0) = 1.252951810896874e-9
(my_graph.n2.z ≥ 0) = 0.0
(my_graph.n3.x[1] ≥ 0) = 3.1323794185248956e-10
(my_graph.n3.x[2] ≥ 0) = 11.999997964035542
(my_graph.n3.x[3] ≥ 0) = 1.252951810896874e-9
(my_graph.n4.x[1] ≥ 0) = 3.1323794185248956e-10
(my_graph.n4.x[2] ≥ 0) = 1.25295187954758e-9
(my_graph.n1.x ≤ 2) = -0.03001344141633
(my_graph.n1.y ≤ 3) = 0.0
(my_graph.n1.z ≤ 2) = -0.8527692550052085
(my_graph.n2.z ≤ 2) = -12.177242136754801
(my_

# Modeling with Subgraphs
The primary capability of Plasmo.jl comes from its abstraction to handle nested optimization structures using subgraphs. Here we show how to build and optimize an optigraph that consists of two subgraphs that represent separate optimization problems.

First we create a convenience function that adds a template optinode model given an optigraph.

In [32]:
function add_node_model(graph::OptiGraph)
	node = add_node(graph)
    @variable(node, x >= 0)
    @variable(node, y >= 1)
    @constraint(node, x + y <= 5)
    @constraint(node, exp(x) >= 2)
    return node
end

add_node_model (generic function with 1 method)

## Create a Top-Level OptiGraph

In [47]:
# the top-level graph
graph = OptiGraph(;name=:top_graph)

An OptiGraph: top_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         0                0
          Edges:         0                0
      Subgraphs:         0                0
      Variables:         0                0
    Constraints:         0                0


## Create Subgraph Models

In [48]:
subgraph1 = OptiGraph(;name=:sub_1)
n1 = add_node_model(subgraph1)
n2 = add_node_model(subgraph1)
@linkconstraint(subgraph1, n1[:x] == n2[:x])

subgraph2 = OptiGraph(;name=:sg2)
n3 = add_node_model(subgraph2)
n4 = add_node_model(subgraph2)
@linkconstraint(subgraph2, n3[:x] == n4[:x])

@show subgraph1
@show subgraph2
;

subgraph1 = An OptiGraph:     sub_1
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         2                2
          Edges:         1                1
      Subgraphs:         0                0
      Variables:         4                4
    Constraints:         9                9

subgraph2 = An OptiGraph:       sg2
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         2                2
          Edges:         1                1
      Subgraphs:         0                0
      Variables:         4                4
    Constraints:         9                9



## Add Subgraphs to Top-Level Graph

In [49]:
add_subgraph(graph, subgraph1)
add_subgraph(graph, subgraph2);
graph

An OptiGraph: top_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         0                4
          Edges:         0                2
      Subgraphs:         2                2
      Variables:         0                8
    Constraints:         0               18


## Add Linking Constraints and Objective Function
Note that these linking constraint connect nodes across subgraphs. Each subgraph can be treated as a stand-alone optimization problem linked within a larger problem.

In [50]:
@linkconstraint(graph, n1[:x] == n3[:x])
@linkconstraint(graph, n2[:x] == n4[:x])
@objective(graph, Min, sum(node[:x] + node[:y] for node in all_nodes(graph)))
graph

An OptiGraph: top_graph
                 #local elements  #total elements
--------------------------------------------------
          Nodes:         0                4
          Edges:         2                4
      Subgraphs:         2                2
      Variables:         0                8
    Constraints:         2               20


## Optimize Subgraphs in Isolation

In [59]:
@objective(subgraph1, Min, sum(node[:x] for node in all_nodes(subgraph1)))
@objective(subgraph2, Min, sum(node[:y] for node in all_nodes(subgraph2)))
set_optimizer(subgraph1, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0))
set_optimizer(subgraph2, optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0))
optimize!(subgraph1)
optimize!(subgraph2)

@show objective_value(subgraph1)
@show objective_value(subgraph2)

objective_value(subgraph1) = 1.3862943561225711
objective_value(subgraph2) = 1.9999999850213606
value.(subgraph1, all_variables(subgraph1)) = [0.6931471780612856, 2.1267984211100464, 0.6931471780612856, 2.1267984211100464]


4-element Vector{Float64}:
 0.6931471780612856
 2.1267984211100464
 0.6931471780612856
 2.1267984211100464

Note that variables and constraints can take on different solution values depending on the graph they are mapped to. In this case, our variables could take on different solutions depending if they are solved in the subgraph or the top-level graph. Consequently, it is always recommended to specify the graph when making calls to `value` or `dual`. By default, these function will return the value from the graph the nodes and edges were defined in.

In [61]:
@show value.(subgraph1, all_variables(subgraph1))
@show value.(subgraph2, all_variables(subgraph2))

value.(subgraph1, all_variables(subgraph1)) = [0.6931471780612856, 2.1267984211100464, 0.6931471780612856, 2.1267984211100464]
value.(subgraph2, all_variables(subgraph2)) = [3.2786254193817546, 0.9999999925106803, 3.2786254193817546, 0.9999999925106803]


4-element Vector{Float64}:
 3.2786254193817546
 0.9999999925106803
 3.2786254193817546
 0.9999999925106803

## Optimize the Full OptiGraph
We lastly optimize the full graph and show the solution. Note that we could do cool things here like use the subgraph solutions to warm-start the full solution. Optigraphs offer flexibility with solution approaches to optimization problems.

In [69]:
set_start_value.(graph, all_variables(subgraph1), value.(subgraph1, all_variables(subgraph1)))
set_start_value.(graph, all_variables(subgraph2), value.(subgraph2, all_variables(subgraph2)))
@show start_value.(graph, all_variables(subgraph1))
@show start_value.(graph, all_variables(subgraph2))

set_optimizer(graph, Ipopt.Optimizer)
optimize!(graph);

start_value.(graph, all_variables(subgraph1)) = [0.6931471780612856, 2.1267984211100464, 0.6931471780612856, 2.1267984211100464]
start_value.(graph, all_variables(subgraph2)) = [3.2786254193817546, 0.9999999925106803, 3.2786254193817546, 0.9999999925106803]
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        8
                     variables with only lower bounds:        8
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        8
        inequality constraints with only lower bounds:        4
   inequality constraints wit

## Query Full Graph Solution

In [68]:
@show objective_value(graph)
@show value.(graph, all_variables(subgraph1))
@show value.(graph, all_variables(subgraph2));

objective_value(graph) = 6.772588682269025
value.(graph, all_variables(subgraph1)) = [0.6931471780603017, 0.9999999925069265, 0.6931471780603017, 0.9999999925069265]
value.(graph, all_variables(subgraph2)) = [0.6931471780603584, 0.9999999925069265, 0.6931471780603584, 0.9999999925069265]
